# General Workbook  

> Using Alhazen in my general day-to-day work.

Note - this question is inherently driven by discussion and informal experience (as opposed to formal experimentation). So we would expect to 

## Preliminaries

In [2]:
from alhazen.aliases import *
from alhazen.core import lookup_chat_models
from alhazen.agent import AlhazenAgent
from alhazen.schema_sqla import *
from alhazen.core import lookup_chat_models
from alhazen.tools.basic import AddCollectionFromEPMCTool, DeleteCollectionTool
from alhazen.tools.paperqa_emulation_tool import PaperQAEmulationTool
from alhazen.tools.metadata_extraction_tool import * 
from alhazen.tools.protocol_extraction_tool import *
from alhazen.tools.tiab_classifier_tool import *
from alhazen.tools.tiab_mapping_tool import *
from alhazen.toolkit import *
from alhazen.utils.jats_text_extractor import NxmlDoc

from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database, backup_ceifns_database
from alhazen.utils.searchEngineUtils import *


from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from langchain_community.chat_models.ollama import ChatOllama
from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI

from bs4 import BeautifulSoup,Tag,Comment,NavigableString
from databricks import sql
from datetime import datetime
from importlib_resources import files
import os
import pandas as pd
from pathlib import Path
import re
import requests

from sqlalchemy import text, create_engine, exists, func, or_, and_, not_, desc, asc
from sqlalchemy.orm import sessionmaker, aliased

from time import time,sleep
from tqdm import tqdm
from urllib.request import urlopen
from urllib.parse import quote_plus, quote, unquote
from urllib.error import URLError, HTTPError
import uuid
import yaml

Remember to set environmental variables for this code:

* `ALHAZEN_DB_NAME` - the name of the PostGresQL database you are storing information into
* `LOCAL_FILE_PATH` - the location on disk where you save temporary files, downloaded models or other data.   

In [4]:
if os.environ.get('LOCAL_FILE_PATH') is None: 
    raise Exception('Where are you storing your local literature database?')
if os.path.exists(os.environ['LOCAL_FILE_PATH']) is False:
    os.makedirs(os.environ['LOCAL_FILE_PATH'])    

loc = os.environ['LOCAL_FILE_PATH']
db_name = 'open_science_best_practices'

In [ ]:
drop_ceifns_database(os.environ['ALHAZEN_DB_NAME'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Database has been backed up to /users/gully.burns/alhazen/alhazen_workbooks/backup2024-02-14-09-25-50.sql
Database has been dropped successfully !!


In [ ]:
create_ceifns_database(os.environ['ALHAZEN_DB_NAME'])

100%|██████████| 311/311 [00:00<00:00, 2249.83it/s]


In [5]:
ldb = Ceifns_LiteratureDb(loc=loc, name=db_name)
llms_lookup = lookup_chat_models()
print(llms_lookup.keys())

dict_keys(['ollama_mixtral', 'gpt4.0', 'gpt3.5', 'gemini1.0', 'databricks_dbrx', 'databricks_mixtral'])


In [8]:
llm_gpt4 = llms_lookup.get('gpt4.0')
llm_mixt = llms_lookup.get('ollama_mixtral')
llm_dbrx = llms_lookup.get('databricks_dbrx')

cb = AlhazenAgent(llm_mixt, llm_gpt4, db_name=db_name)
print('AGENT TOOLS')
for t in cb.tk.get_tools():
    print('\t'+type(t).__name__)

AGENT TOOLS
	AddCollectionFromEPMCTool
	AddAuthorsToCollectionTool
	DescribeCollectionCompositionTool
	DeleteCollectionTool
	RetrieveFullTextTool
	RetrieveFullTextToolForACollection
	MetadataExtraction_EverythingEverywhere_Tool
	SimpleExtractionWithRAGTool
	PaperQAEmulationTool
	ProcotolEntitiesExtractionTool
	CheckExpressionTool
	TitleAbstractClassifier_OneDocAtATime_Tool
	TitleAbstractDiscourseMappingTool


In [10]:
t = [t for t in cb.tk.get_tools() if isinstance(t, AddCollectionFromEPMCTool)][0]
#t.run({'id': '0', 
#       'name':'Best Practices for Data and Model Sharing', 
#       'query':'\"best practices\" AND (\"data sharing\" OR \"model sharing\")'})
t.run({'id': '1', 
       'name':'Best Practices for Machine Learning Model Sharing', 
       'query':'"best practices" AND "model sharing" AND "machine learning"'})

https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query="best practices" AND "model sharing" AND "machine learning", 33 European PMC PAPERS FOUND


100%|██████████| 1/1 [00:19<00:00, 19.55s/it]


 Returning 27


100%|██████████| 27/27 [00:17<00:00,  1.56it/s]
/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:293: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


{'response': 'We added a collection to the database called `Best Practices for Machine Learning Model Sharing` containing 0 papers from this query: `"best practices" AND "model sharing" AND "machine learning"`.'}